In [13]:
import numpy as np
import cvxpy as cp

def fit_plrt(X, Y, r, M, F):
    # Ensure Y is a column vector
    Y = Y.reshape(-1, 1)
    n, p = X.shape
    m = len(F)
    
    # Adjust indices in F from MATLAB (1-based) to Python (0-based)
    F = [np.array(f) - 1 for f in F]
    
    # Compute rho and cum_rho
    rho = np.zeros(m, dtype=int)
    for k in range(m):
        rho[k] = np.prod(r[F[k]])
    cum_rho = np.hstack(([0], np.cumsum(rho)))
    
    nvars = cum_rho[-1] + n + 2 * m
    
    # Variables
    U = cp.Variable(cum_rho[-1])
    pX_U = cp.Variable(n)
    eta = cp.Variable(m)
    nu = cp.Variable(m)
    
    # Objective function
    obj = cp.Minimize(cp.sum((1 / n) * cp.exp(pX_U) - cp.multiply(Y.flatten() / n, pX_U)))
    
    # Constraints
    constraints = []
    
    # Lower bounds on eta
    constraints += [eta >= -2 * np.log(M)]
    # Upper bounds on nu
    constraints += [nu <= 2 * np.log(M)]
    
    # Constraints: U >= eta[ind]
    for ind in range(m):
        constraints += [U[cum_rho[ind]:cum_rho[ind+1]] - eta[ind] >= 0]
        constraints += [nu[ind] - U[cum_rho[ind]:cum_rho[ind+1]] >= 0]
    
    # Sum constraints
    constraints += [cp.sum(eta) + np.log(M) >= 0]
    constraints += [np.log(M) - cp.sum(nu) >= 0]
    
    # Equality constraints: pX_U = sum(U at appropriate indices)
    for i in range(n):
        expr = 0
        for k in range(m):
            Fk = F[k]
            if len(Fk) > 1:
                sub_indices = X[i, Fk] - 1  # Adjust for zero-based indexing
                dims = r[Fk]
                idx = np.ravel_multi_index(sub_indices.astype(int), dims.astype(int))
                expr += U[cum_rho[k] + idx]
            else:
                idx = int(X[i, Fk[0]] - 1)
                expr += U[cum_rho[k] + idx]
        constraints += [pX_U[i] == expr]
    
    # Solve the problem using MOSEK
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    U_value = U.value
    pobjval = prob.value
    
    return U_value, pobjval


In [14]:
X = np.array([[1, 1, 1],
              [2, 1, 3],
              [2, 1, 2],
              [2, 2, 3],
              [3, 1, 3],
              [2, 3, 3]])

Y = np.array([0.5, 0.6, 0.3, 0.7, 0.3, 0.8])

# The maximum value for each predictor (assuming predictors range from 1 to r_j)
r = np.max(X, axis=0)
r

array([3, 3, 3])

In [15]:
import numpy as np

# Sample data for testing the fit_plrt function

# Predictor matrix X (each row is a sample, each column is a predictor)
X = np.array([[1, 1, 1],
              [2, 1, 3],
              [2, 1, 2],
              [2, 2, 3],
              [3, 1, 3],
              [2, 3, 3]])

Y = np.array([0.5, 0.6, 0.3, 0.7, 0.3, 0.8])

# The maximum value for each predictor (assuming predictors range from 1 to r_j)
r = np.max(X, axis=0)

# Scalar parameter M
M = 4

# Grouping structure F (list of arrays)
# Example 1: Each predictor is its own group
F = [np.array([1]), np.array([2]), np.array([3])]

# Example 2: Group first two predictors together
# F = [np.array([1, 2]), np.array([3])]

# Call the fit_plrt function
U_value, pobjval = fit_plrt(X, Y, r, M, F)

# Display the outputs
print('Optimized U values:')
print(U_value)

print('Objective function value:')
print(pobjval)


Optimized U values:
[-0.119485   -0.08611939 -0.46606811 -0.45415308 -0.18447056 -0.05092965
 -0.11948935 -0.46607239 -0.08612421]
Objective function value:
0.8366709194928581


In [12]:
F = [np.array([1]), np.array([2]), np.array([3])]
F

[array([1]), array([2]), array([3])]

In [5]:
print('Optimized U values:')
print(U_value)

print('Objective function value:')
print(pobjval)


Optimized U values:
[ 0.46935075  0.56051166 -0.13263572  0.15991074  0.26530034  0.26530924
  0.46935075 -0.13263572  0.56051166]
Objective function value:
-0.7166527296712855
